In this notebook, I apply random forest (with parameter class_weight = 'balanced') to the data set with binarized labels ('NDF' and 'non-NDF').

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#from xgboost.sklearn import XGBClassifier

np.random.seed(0)

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn
import sklearn.grid_search
#from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

import math
from sklearn.metrics import confusion_matrix
from NDCG_score_func import ndcg_score
from __future__ import division

Loading X,y and X_test

In [2]:
X= np.load('X.npy')
y= np.load('y.npy')
X_test= np.load('X_test.npy')

In [3]:
y = y.astype(int)
y = np.equal(y, np.ones((len(y)))*7)
y = np.invert(y)
#y
#booked-1; NDF-0

Let's first split the origitnal traiing data into training set and test set.

In [4]:
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X,y, test_size=0.25, random_state=42)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(160088L, 226L)
(53363L, 226L)
(160088L,)
(53363L,)


In [5]:
recip_freq = len(y_test) / 2 *np.bincount(y_test.astype(int)).astype(np.float64)
print recip_freq

[  8.28167078e+08   5.95637806e+08]


### Random Forest

Fit random forest classifier with training set and test the performance on test set.

In [6]:
clf = RandomForestClassifier(max_features=40, n_estimators=120, n_jobs=1, min_samples_split=5,class_weight = 'balanced')#n_jobs=2 #, min_samples_split=5  max_features=12
clf.fit(X_train,y_train)
y_test_pred = clf.predict(X_test)
Labels = ['NDF','Booked']
confusion= pd.DataFrame(data = confusion_matrix(y_test, y_test_pred),index = Labels, columns=Labels)

__Confusion Matrix__

In the following confusion matrix, the row index represent the true book type, and the columns index represents the predicted booking class.

In [7]:
confusion['total']=confusion.sum(axis=1)
tem = pd.Series(data = [ confusion.iloc[0,0]/ confusion.ix['NDF','total'],\
                         confusion.iloc[1,1]/ confusion.ix['Booked','total'] ], index = Labels)
confusion['recognized']=tem
confusion

,NDF,Booked,total,recognized
NDF,22844,8195,31039,0.735977
Booked,7724,14600,22324,0.654005


This time, the result is much more balanced compared to the case with non-binarized lables.

__Accuracy__ (just for recording)

In [8]:
sklearn.metrics.accuracy_score(y_test, y_test_pred)

0.70168468789235983